## Deploying the Production Model
The production model has been saved and will be imported in this file to simulate its deployment at the supermarket.

In [1]:
# import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from PIL import Image, ImageOps

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import load_model

# import the opencv library to access the camera
import cv2

### Define Functions

In [2]:
# This function presents the top-2 predictions, together with similar vegetables if any, for the consumer to choose from
def selectFromDict(pred):
    inputValid = False
    veg_list = []
    while not inputValid:
        print('Is it:')
        count = 0
        for idx in range(2):
            count += 1
            veg_list.append(pred.index[idx])
            #veg_list.append(pred.index[idx] + f' ({np.round(pred.values[idx]*100,0)}%)')
            
            # Are there any similar vegetable
            if similar_df.loc[pred.index[idx],'similar']!="none":
                veg_list.append(similar_df.loc[pred.index[idx],'similar'])
        
        # List vegetables options
        for idx in range(len(veg_list)):
            print(f'{idx+1}) {veg_list[idx]}')
        
        # Add a scan again option
        scan_again = len(veg_list) + 1
        print(f'{len(veg_list)+1}) Scan again')
                  
        try:
            choice = int(input('Choose one: '))
            if choice==scan_again:
                selected = ""
                inputValid = True
            elif 0<choice<scan_again:
                selected = veg_list[choice-1]
                inputValid = True
            else:
                clear_output(wait=True)
        except:
            clear_output(wait=True)
    
    clear_output(wait=True)        
    return selected

# This function prints the label upon confirming the choice
def print_label(veg, weight):
    price_per_kg = price_df.loc[veg,'price_per_kg']
    total_price = price_per_kg * weight 
    print(veg + '\n' + '-' * 30)
    print(f'PRICE/KG: ${price_per_kg}')
    print(f'WEIGHT: {np.round(weight,3)} kg')
    print(f'TOTAL PRICE: ${np.round(total_price,2)}')

### Load the Production Model
The production model, VGG16 and imagenet weights and vegetable classes are loaded.

In [7]:
# load production model
model_prod = load_model('./models/prod_model.hdf5')

# Load imagenet weights 
conv_base = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(100, 100, 3))  # 3 = number of channels in RGB pictures

# Load the classes
infile = open('./models/veg_class.pkl','rb')
veg_df = pickle.load(infile)
infile.close()

# Load the vegetable prices
price_df = pd.read_csv('./datasets/price_per_kg.csv', index_col=0)

# Load the similar vegetables file
similar_df = pd.read_csv('./datasets/similar_vegetables.csv', index_col=0)

### Simulating the Deployment
The picture below shows how a weighing scale is placed at the supermarket.

<img src="./images_for_notebook/scale.jpg" alt="Weighing scale at NTUC" width="250" align="left" style="margin:5px 50px"/>

For our deployment, a pinhole camera can be attached to the weighing scale, looking down vertically on the items placed on the scale. The process should be as follows:

1. First, place the item on the scale.


2. Upon detecting the change in weight, the scale will activate the camera to do a scan.


3. An image is taken and passed to the model for prediction.


4. The top-2 predicted items can be presented for the consumer to select from.


5. Upon confirming the item, a label depicting the item description, price per kg and weight will be printed.

In [8]:
### Start the camera
cam = cv2.VideoCapture(0)

size = (100,100)
font = cv2.FONT_HERSHEY_DUPLEX

tag1 = ""
tag2 = ""

while True:
    
    ret, frame = cam.read()
    # OpenCV uses BGR as its default colour order for images, convert frame back to RGB for display and prediction
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    cv2.putText(frame, "1. Press <Space> to Scan",(10, 10),font,fontScale=0.4,color=(0,255,0))
    cv2.putText(frame, "2. Press <Esc> to End",(10, 25),font,fontScale=0.4,color=(0,255,0))
    
    k = cv2.waitKey(10)

    if k==32:
        # SPACE pressed                
        h,w = frame_rgb.shape[:2]

        # Crop to square and resize to 100x100
        if w>h:
            crop = int(np.round((w-h)/2,0))
            pic = frame_rgb[0:h, crop:w-crop]
            pic = cv2.resize(pic,size)
        elif w==h:
            pic = cv2.resize(frame_rgb,size)
        else:
            crop = int(np.round((h-w)/2,0))
            pic = frame_rgb[crop:h-crop, 0:w]
            pic = cv2.resize(pic,size)
            
        pic = np.array(pic)/255
        
        # Display the captured image used for prediction
        plt.imshow(pic)
        plt.show()

        pic = pic.reshape(1,100,100,3)
        
        # Extract features for image
        features = conv_base.predict(pic)
        pred = pd.Series(model_prod.predict(features)[0], index=veg_df.index).sort_values(ascending=False)[:2]
        pred_class = pred.index[0]
        pred_prob = pred.values[0]

        # Show label on image frame
        tag1 = f'{pred.index[0]}' + f'({np.round(pred.values[0]*100,0)}%)'
        tag2 = f'{pred.index[1]}' + f'({np.round(pred.values[1]*100,0)}%)'
        
        veg = selectFromDict(pred)
        clear_output(wait=True)
        if veg!="":
            print_label(veg, np.random.rand())
        
    if k==27:
        # ESC pressed
        break
    
    # Show the predictions on screen
    # cv2.putText(frame, tag1,(10, 55),font,fontScale=0.7,color=(0,255,0))
    # cv2.putText(frame, tag2,(10, 75),font,fontScale=0.7,color=(0,255,0))
    cv2.imshow("test", frame)

# Once the camera is released, close the image
cam.release()
cv2.destroyAllWindows()

broccoli
------------------------------
PRICE/KG: $7.57
WEIGHT: 0.793 kg
TOTAL PRICE: $6.0


In [45]:
# Run this in case camera becomes unresponsive
cam.release()
cv2.destroyAllWindows()